# This notebook serves for the purpose of testing code
# The content will be transfered to run.py for submission and evaluation

In [22]:
import numpy as np
import time
from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator
from flatland.envs.rail_env import RailEnv
from flatland.utils.rendertools import RenderTool
from flatland.envs.observations import GlobalObsForRailEnv
from flatland.envs.observations import TreeObsForRailEnv
from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_env import GridTransitionMap


### Not moving to the run.py


In [47]:
NUMBER_OF_AGENTS = 10

nr_start_goal = 10
nr_extra = 2
min_dist = 8
max_dist = 99999
seed = 0
# seed = np.random.randint(0,1000)

### Not moving to the run.py


In [48]:
my_rail_generator = complex_rail_generator (
        nr_start_goal,
        nr_extra,
        min_dist,
        max_dist,
        seed
    )

env = RailEnv(
    
    # parameters taht control the dimensions of the map.
    width = 30,
    height = 30,
    
    # Rail generator: generates new rail networks on each reset
    # simplest rail generators: 
    #         envs.rail_generators.rail_from_manual_specifications_generator and 
    #         envs.rail_generators.random_rail_generator.
    
    rail_generator = my_rail_generator,
    schedule_generator = complex_schedule_generator(),
    obs_builder_object= GlobalObsForRailEnv(),
    number_of_agents = NUMBER_OF_AGENTS
)


### move to run.py


In [49]:
# The original railway map from the complex_rail_generator
original_rail_map = env.rail.grid.copy()




### Move to run.py

In [50]:
# straightward implementation, could be optimized
# The node id is in the form (x,y), could be a linear mapping from 
# the most leftupper cornner to the most rightlower corner

# Flatland map encoding scheme, NESW -> N:[NESW], E:[NESW], S[NESW], W[NESW]
# Deadend cell is different, agent faces the opposite to the direction that it can goes. 


class decode_map:
    
    
    def __init__(self,original_map):
        self.original_map = original_map
        self.converted_input = {}

    # Determine next nodes for current_node.
    def case_matching(self, current_node, short_bits):
        
        next_nodes = []
        
        if (int(short_bits[0]) == 1):
            
            next_node = (current_node[0]-1, current_node[1])
#             print("N", next_node)
            
            next_nodes.append(next_node)
        
        if(int(short_bits[1]) == 1):
            
            next_node = (current_node[0], current_node[1]+1)
#             print("E", next_node)
            
            next_nodes.append(next_node)

        
        if(int(short_bits[2]) == 1):
            
            next_node = (current_node[0]+1, current_node[1])
#             print("S", next_node)
            
            next_nodes.append(next_node)

        if(int(short_bits[3]) == 1):
            
            next_node = (current_node[0], current_node[1]-1)
#             print("W", next_node)
            
            next_nodes.append(next_node)
        
        return next_nodes
        
    # First find the possible previous nodes, and call another function to find the possible next nodes.
    def slice_16_bits(self, current_node, bits):
        
        previous_nodes = {}
        
        # Facing North
        
        if (bits[0:4] != "0000"):
            
#             print(bits[0:4])
            next_nodes = self.case_matching(current_node, bits[0:4])
#             print("N, and returned next nodes", next_nodes)
            
            # add one previous node, and what nodes the agent can go from current node.
            previous_nodes[(current_node[0]+1,current_node[1])] = next_nodes
            
#             print("N, and current previous nodes", previous_nodes)
            
        # East
        if (bits[4:8] != "0000"):

#             print(bits[4:8])
            next_nodes = self.case_matching(current_node, bits[4:8])
#             print("E",next_nodes)
            
            previous_nodes[(current_node[0], current_node[1]-1)] = next_nodes
            
#             print("E, and current previous nodes", previous_nodes)
# 

        # South
        if (bits[8:12] != "0000"):
            
#             print(bits[8:12])
            next_nodes = self.case_matching(current_node, bits[8:12])
#             print("S",next_nodes)
            
            previous_nodes[(current_node[0]-1, current_node[1])] = next_nodes
            
#             print("S, and current previous nodes", previous_nodes)
       
        # West
        if (bits[12:16] != "0000"):
            
#             print(bits[12:16])
            next_nodes = self.case_matching(current_node, bits[12:16])
#             print("W",next_nodes)
            
            previous_nodes[(current_node[0], current_node[1]+1)] = next_nodes
            
#             print("W, and current previous nodes", previous_nodes)

        return previous_nodes
            
    def convert_ori_rail_map(self):
        for row in range(0,len(self.original_map)):
            for col in range(0,len(self.original_map[row])):
                
                # Current node: (row,col)
                print("Current node:", (row,col))
                current_node = (row,col)
                # Get previous node, decided by which direction the agent is facing.
                # EXCEPT the deadend nodes.
                
                
#                 print('{0:016b}'.format(self.original_map[row][col]))
                previous_next_nodes = self.slice_16_bits(current_node, '{0:016b}'.format(self.original_map[row][col]))
#                 print(previous_next_nodes.items())
                
                for previous,direction in previous_next_nodes.items():
#                     print("direction", direction)

                    self.converted_input[(current_node, previous)] = direction
#         print(self.converted_input)
        return self.converted_input

### Move to run.py main loop.

In [51]:
result = decode_map(original_rail_map).convert_ori_rail_map()

Current node: (0, 0)
Current node: (0, 1)
Current node: (0, 2)
Current node: (0, 3)
Current node: (0, 4)
Current node: (0, 5)
Current node: (0, 6)
Current node: (0, 7)
Current node: (0, 8)
Current node: (0, 9)
Current node: (0, 10)
Current node: (0, 11)
Current node: (0, 12)
Current node: (0, 13)
Current node: (0, 14)
Current node: (0, 15)
Current node: (0, 16)
Current node: (0, 17)
Current node: (0, 18)
Current node: (0, 19)
Current node: (0, 20)
Current node: (0, 21)
Current node: (0, 22)
Current node: (0, 23)
Current node: (0, 24)
Current node: (0, 25)
Current node: (0, 26)
Current node: (0, 27)
Current node: (0, 28)
Current node: (0, 29)
Current node: (1, 0)
Current node: (1, 1)
Current node: (1, 2)
Current node: (1, 3)
Current node: (1, 4)
Current node: (1, 5)
Current node: (1, 6)
Current node: (1, 7)
Current node: (1, 8)
Current node: (1, 9)
Current node: (1, 10)
Current node: (1, 11)
Current node: (1, 12)
Current node: (1, 13)
Current node: (1, 14)
Current node: (1, 15)
Current 

### Experimental part, not moving to run.py

In [52]:
# pretty print result

for i in result.items():
    print(i,"\n")

(((0, 18), (1, 18)), [(1, 18)]) 

(((1, 9), (2, 9)), [(2, 9)]) 

(((1, 18), (2, 18)), [(0, 18)]) 

(((1, 18), (0, 18)), [(2, 18)]) 

(((2, 9), (3, 9)), [(1, 9)]) 

(((2, 9), (1, 9)), [(3, 9)]) 

(((2, 18), (3, 18)), [(1, 18)]) 

(((2, 18), (1, 18)), [(3, 18)]) 

(((3, 7), (3, 8)), [(3, 8)]) 

(((3, 8), (3, 7)), [(3, 9)]) 

(((3, 8), (3, 9)), [(3, 7)]) 

(((3, 9), (4, 9)), [(2, 9)]) 

(((3, 9), (3, 8)), [(3, 10)]) 

(((3, 9), (2, 9)), [(4, 9)]) 

(((3, 9), (3, 10)), [(3, 8)]) 

(((3, 10), (3, 9)), [(3, 11)]) 

(((3, 10), (3, 11)), [(3, 9)]) 

(((3, 11), (3, 10)), [(3, 12)]) 

(((3, 11), (3, 12)), [(3, 10)]) 

(((3, 12), (3, 11)), [(3, 13)]) 

(((3, 12), (3, 13)), [(3, 11)]) 

(((3, 13), (3, 12)), [(3, 14)]) 

(((3, 13), (3, 14)), [(3, 12)]) 

(((3, 14), (3, 13)), [(3, 15)]) 

(((3, 14), (3, 15)), [(3, 13)]) 

(((3, 15), (3, 14)), [(3, 16)]) 

(((3, 15), (3, 16)), [(3, 14)]) 

(((3, 16), (3, 15)), [(3, 17)]) 

(((3, 16), (3, 17)), [(3, 15)]) 

(((3, 17), (3, 16)), [(3, 18)]) 

(((3, 17),

### Experimental part, not moving to run.py

In [53]:
# Obtain the start and target locations
# Assuming that each agent is going from one assigned start location to one fixed target location.
# i.e. no multiple targets available for one agent to go to.

# can be access by env.agents (lists of agents objects with their own information)
'''
informaiton include: poistion (i.e. start location at time step 0), 
                     direction,
                     target,
                     speed_data,
                     malfunction_data,
                     handle,
                     old_direction,
                     old_position     
'''

agent_info = {}

for index, agent in enumerate(env.agents):
    print(index, agent)
   

0 EnvAgent(position=(12, 15), direction=1, target=(21, 0), moving=False, speed_data={'position_fraction': 0.0, 'speed': 1.0, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunction_rate': 0.0, 'next_malfunction': 0, 'nr_malfunctions': 0}, handle=0, old_direction=None, old_position=None)
1 EnvAgent(position=(3, 27), direction=1, target=(3, 7), moving=False, speed_data={'position_fraction': 0.0, 'speed': 1.0, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunction_rate': 0.0, 'next_malfunction': 0, 'nr_malfunctions': 0}, handle=1, old_direction=None, old_position=None)
2 EnvAgent(position=(9, 19), direction=0, target=(21, 18), moving=False, speed_data={'position_fraction': 0.0, 'speed': 1.0, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunction_rate': 0.0, 'next_malfunction': 0, 'nr_malfunctions': 0}, handle=2, old_direction=None, old_position=None)
3 EnvAgent(position=(24, 12), direction=0, targe

### Move to run.py

In [54]:
# idx2node: {index:(current, previous)}
idx2node = {idx:k for idx, k in enumerate(result.keys())} 

# node2idx: {(current, previous) : index}
node2idx = {k:idx for idx, k in idx2node.items()}

# idx2pos: {index:current}
idx2pos = {k:v[0] for (k,v) in idx2node.items()}

edges = []
for cur, prev in result:
    print (cur, prev, result[(cur, prev)])
    for to in result[(cur, prev)]:
        print()
        print(node2idx[(to, cur)])
        edges.append((node2idx[(cur, prev)], node2idx[(to, cur)]))
        
        
def linearize_loc(env, loc):
    return loc[0]*env.width + loc[1]

(0, 18) (1, 18) [(1, 18)]

3
(1, 9) (2, 9) [(2, 9)]

5
(1, 18) (2, 18) [(0, 18)]

0
(1, 18) (0, 18) [(2, 18)]

7
(2, 9) (3, 9) [(1, 9)]

1
(2, 9) (1, 9) [(3, 9)]

13
(2, 18) (3, 18) [(1, 18)]

2
(2, 18) (1, 18) [(3, 18)]

33
(3, 7) (3, 8) [(3, 8)]

9
(3, 8) (3, 7) [(3, 9)]

12
(3, 8) (3, 9) [(3, 7)]

8
(3, 9) (4, 9) [(2, 9)]

4
(3, 9) (3, 8) [(3, 10)]

15
(3, 9) (2, 9) [(4, 9)]

60
(3, 9) (3, 10) [(3, 8)]

10
(3, 10) (3, 9) [(3, 11)]

17
(3, 10) (3, 11) [(3, 9)]

14
(3, 11) (3, 10) [(3, 12)]

19
(3, 11) (3, 12) [(3, 10)]

16
(3, 12) (3, 11) [(3, 13)]

21
(3, 12) (3, 13) [(3, 11)]

18
(3, 13) (3, 12) [(3, 14)]

23
(3, 13) (3, 14) [(3, 12)]

20
(3, 14) (3, 13) [(3, 15)]

25
(3, 14) (3, 15) [(3, 13)]

22
(3, 15) (3, 14) [(3, 16)]

27
(3, 15) (3, 16) [(3, 14)]

24
(3, 16) (3, 15) [(3, 17)]

29
(3, 16) (3, 17) [(3, 15)]

26
(3, 17) (3, 16) [(3, 18)]

32
(3, 17) (3, 18) [(3, 16)]

28
(3, 18) (4, 18) [(2, 18)]

6
(3, 18) (3, 17) [(3, 19)]

36
(3, 18) (2, 18) [(4, 18)]

62
(3, 18) (3, 19) [(3,

In [55]:
idx2node

{0: ((0, 18), (1, 18)),
 1: ((1, 9), (2, 9)),
 2: ((1, 18), (2, 18)),
 3: ((1, 18), (0, 18)),
 4: ((2, 9), (3, 9)),
 5: ((2, 9), (1, 9)),
 6: ((2, 18), (3, 18)),
 7: ((2, 18), (1, 18)),
 8: ((3, 7), (3, 8)),
 9: ((3, 8), (3, 7)),
 10: ((3, 8), (3, 9)),
 11: ((3, 9), (4, 9)),
 12: ((3, 9), (3, 8)),
 13: ((3, 9), (2, 9)),
 14: ((3, 9), (3, 10)),
 15: ((3, 10), (3, 9)),
 16: ((3, 10), (3, 11)),
 17: ((3, 11), (3, 10)),
 18: ((3, 11), (3, 12)),
 19: ((3, 12), (3, 11)),
 20: ((3, 12), (3, 13)),
 21: ((3, 13), (3, 12)),
 22: ((3, 13), (3, 14)),
 23: ((3, 14), (3, 13)),
 24: ((3, 14), (3, 15)),
 25: ((3, 15), (3, 14)),
 26: ((3, 15), (3, 16)),
 27: ((3, 16), (3, 15)),
 28: ((3, 16), (3, 17)),
 29: ((3, 17), (3, 16)),
 30: ((3, 17), (3, 18)),
 31: ((3, 18), (4, 18)),
 32: ((3, 18), (3, 17)),
 33: ((3, 18), (2, 18)),
 34: ((3, 18), (3, 19)),
 35: ((3, 19), (4, 19)),
 36: ((3, 19), (3, 18)),
 37: ((3, 19), (3, 20)),
 38: ((3, 20), (3, 19)),
 39: ((3, 20), (3, 21)),
 40: ((3, 21), (3, 20)),
 41: 

### move to run.py; encapsulate into method

In [37]:
# convert agent start information from the format "start_position, direction = a number"
# to edge

start_idx = []
for agent in env.agents:
    print(agent.position)
    print(agent.direction) # direction encoding: {0: North, 1: East, 2: South, 3: West}
    current_pos = agent.position
    if(agent.direction ==0):
        prev_pos = (current_pos[0]+1,current_pos[1])
    elif(agent.direction == 1):
        prev_pos = (current_pos[0],current_pos[1]-1)
    elif(agent.direction == 2):
        prev_pos = (current_pos[0]-1,current_pos[1])
    elif(agent.direction == 3):
        prev_pos = (current_pos[0], current_pos[1]+1)
    start_node = (current_pos,prev_pos)
    index = node2idx[start_node]
    
    start_idx.append(index)
#     print(start_node, index)

print(start_idx)

(12, 15)
1
(3, 27)
1
(9, 19)
0
(24, 12)
0
(6, 7)
3
(24, 17)
3
(19, 16)
1
(15, 15)
2
(7, 0)
2
(25, 0)
3
[169, 52, 124, 363, 85, 364, 305, 214, 96, 369]


### move to run.py as utilty

In [38]:
pos2nodes = dict()
for _, (cur, prev) in idx2node.items():
    if cur not in pos2nodes:
        pos2nodes[cur] = []
    pos2nodes[cur].append((cur, prev))
    
target_idx = []
for i,agent in enumerate(env.agents):
    goal_nodes = pos2nodes[agent.target]
    # print(goal_nodes)
    if len(goal_nodes) > 1:
        print("Goal node is not a deadend")
    
    target_idx.append(node2idx[goal_nodes[0]])

### Move to run.py as utility

In [12]:
# map file

node_file = ""
edge_file = ""


node_file += str(len(idx2node)) + "\n"
for idx,node in idx2node.items():
    node_file += str(linearize_loc(env, idx2pos[idx])) + "\n"

edge_file += str(len(edges)) + "\n"
for edge in edges:
    edge_file += str(edge[0]) + "," + str(edge[1]) +"\n"

map_file = node_file + edge_file

file = open("map.txt", "w") 
file.write(map_file) 
file.close() 

In [13]:
# agent file
start_goal_pos_file = ""
start_goal_pos_file += str(len(env.agents)) + "\n"
for i,agent in enumerate(env.agents):
    start_goal_pos_file +=  str(start_idx[i]) + "," + str(target_idx[i]) +",1\n"
    

file = open("agents.txt", "w") 
file.write(start_goal_pos_file) 
file.close()

### Experimental code, no longer needed

In [ ]:
# create local files


node_file = ""
current_node_file = ""
edge_file = ""
start_goal_pos_file = ""

node_file += str(len(idx2node)) + "\n"
for idx,node in idx2node.items():
    node_file += str(idx)+ "," + str(node) + "\n"

current_node_file += str(len(idx2node)) + "\n"
for idx,node in idx2node.items():
    current_node_file += str(idx)+ "," + str(node[0]) + "\n"
    
edge_file += str(len(edges)) + "\n"
for edge in edges:
    edge_file += str(edge) +"\n"

start_goal_pos_file += str(len(env.agents)) + "\n"
for i,agent in enumerate(env.agents):
    start_goal_pos_file += str(i) + "," + str(start_idx[i]) + ", " + str(agent.target) +"\n"
    
    
file = open("idx_node.txt", "w") 
file.write(node_file) 
file.close() 

file = open("idx_curr_node.txt", "w") 
file.write(current_node_file) 
file.close()

file = open("edges.txt", "w") 
file.write(edge_file) 
file.close()


file = open("start_goals.txt", "w") 
file.write(start_goal_pos_file) 
file.close()

### Not needed

In [ ]:
# tool to render environments
env_renderer = RenderTool(env, gl="PIL")
# env_renderer = RenderTool(env)

# env_renderer = RenderTool (env)
env_renderer.render_env(show=True)



### Experimental code, not needed.

In [14]:
import os
nowtime = os.popen('g++ ./main.cpp')
result = os.popen('./a.out')
print (result.read())

In [ ]:
import threading

def work1():
    os.popen('g++ ./main.cpp -o a.out')
    result = os.popen('./a.out')
    print(result.read())
def work2():
    os.popen('g++ ./main.cpp -o a1.out')
    result = os.popen('./a1.out')
    print(result.read())
    return result 
    
t1 = threading.Thread(target=work1)
t2 = threading.Thread(target=work2)

t1.start()
t2.start()

In [ ]:
import threading
 
def work1: 
    for i in range(x,y):
        print(i)
 
ta = threading.Thread(target=threadfun,args=(1,6))      
tb = threading.Thread(target=threadfun,args=(10,15))    
ta.start()          


In [ ]:
def my_controller():
    """
    You are supposed to write this controller
    """
    _action = {}
    for _idx in range(NUMBER_OF_AGENTS):
        _action[_idx] = np.random.randint(0, 5)
    return _action

for step in range(1):

    # Get agents' handles to give actions
    # handles = env.get_agent_handles()
    
    # e.g. giving two agents actions. 
    # action_dict = {handles[0]:0, handles[1]:0}
    
    _action = my_controller()
    
    # obs, all_rewards, done: dictionary indexed by agents handles
        # values: correspond to the relevant obeservations, rewards and terminal 
        #       status for each agent.
        
    obs, all_rewards, done, _ = env.step(_action) # environment take one step with the provided actions.
    
    # show results
    print("Observation: \n", obs)
    print("Rewards: {}, [done={}]".format( all_rewards, done))
    env_renderer.render_env(show=True, frames=False, show_observations=False)
    time.sleep(1)

### Rewrite in run.py

In [15]:
import os
import subprocess
print(os.popen("cmake -S ./ECBS -B ./ECBS/build").read())
print(os.popen("make -C ./ECBS/build").read())


-- Configuring done
-- Generating done
-- Build files have been written to: /Users/yizheng/Documents/GitHub/Flatland/Round 1/ECBS/build

[100%] Built target ECBS



In [16]:
result = os.popen("./ECBS/build/ECBS -m ./map.txt  -a ./agents.txt -o ./paths.txt -t 60 -w 1.2 --makespan 1000")
# result = os.popen()
print (result.read())

ECBS: 
        Succeed ; cost: 206, ;lowerbound: 206; preTime: 0.003186; runtime: 1e-05; nodes:   0



### Utility for reading paths, move to run.py 

In [17]:
with open("paths.txt", "r") as f:
    docs = f.readlines()
    
def parse_line_path(l):
    return [int(node) for node in l.split(",")[:-1]]

paths = [parse_line_path(l) for l in docs]

In [ ]:
idx2node[paths[0][0]]

In [ ]:
start_idx

In [ ]:
target_idx

In [18]:
# check connectivity
for path in paths:
    locations = [idx2node[p] for p in path]
#     print(locations)
    for i in range(len(locations) - 1):
        if locations[i][0] != locations[i + 1][1]:
            print("error")

In [19]:
location_paths = [[idx2pos[p] for p in path] for path in paths]

In [20]:
for i in range(len(location_paths) - 1):
    for j in range(i + 1,len(location_paths)):
        for p1, p2 in zip(location_paths[i], location_paths[j]):
            if (p1 == p2):
                print("error")

In [ ]:
location_paths

In [21]:
# pretty print paths

for path in location_paths:
    print(str(path) + "\n")

[(11, 17), (10, 17), (10, 18), (10, 19), (10, 20), (9, 20), (9, 21), (9, 22), (8, 22), (7, 22), (6, 22), (6, 23)]

[(29, 11), (29, 10), (28, 10), (27, 10), (26, 10), (25, 10), (25, 9), (24, 9), (23, 9), (22, 9), (21, 9), (20, 9), (19, 9), (18, 9), (17, 9), (16, 9), (15, 9)]

[(13, 8), (12, 8), (11, 8), (10, 8), (9, 8), (8, 8), (7, 8), (6, 8), (5, 8), (4, 8)]

[(21, 28), (21, 27), (21, 26), (21, 25), (22, 25), (22, 24), (22, 23), (22, 22), (22, 21), (22, 20), (22, 19), (22, 18), (22, 17), (22, 16), (22, 15), (22, 14), (22, 13), (22, 12), (22, 11), (23, 11), (23, 10), (23, 9), (23, 8), (24, 8), (25, 8), (25, 7), (26, 7), (26, 6), (27, 6), (27, 5), (27, 4), (27, 3), (27, 2), (28, 2), (28, 1), (28, 0)]

[(14, 5), (14, 6), (14, 7), (14, 8), (14, 9), (14, 10), (13, 10), (13, 11), (12, 11), (12, 12), (12, 13), (12, 14), (12, 15), (12, 16), (12, 17), (12, 18), (12, 19), (12, 20), (12, 21), (12, 22), (12, 23), (12, 24), (12, 25), (12, 26), (12, 27), (12, 28), (12, 29)]

[(8, 17), (9, 17), (9, 1

In [ ]:
## Convert paths to actions

def paths_to_actions(paths):
    actions_agent = {}
    for i,path in enumerate(paths):
        actions_agent[i] = []
        
        
def location2action(first_node, second_node):
    

## Experimental code


In [ ]:
lo

In [ ]:
paths